# Амангелді Нұрғалым - Зертханалық жұмыс- BERT, СИБ 23-01

### Этот код импортирует необходимые библиотеки и задает пути к файлам, необходимым для загрузки предварительно обученной модели BERT.

In [1]:
import sys
import codecs
import numpy as np
from keras_bert import load_trained_model_from_checkpoint
import tokenization

# папка, куда распаковали предобученную нейросеть BERT
folder = 'multi_cased_L-12_H-768_A-12'

config_path = folder+'/bert_config.json'
checkpoint_path = folder+'/bert_model.ckpt'
vocab_path = folder+'/vocab.txt'

### Таким образом, в этом коде создается токенизатор на основе словаря BERT, а затем загружается предварительно обученная модель BERT с использованием указанных конфигурационного файла и файла контрольных точек.

In [2]:
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_path, do_lower_case=False)

In [3]:
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
model.summary()

C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input-Token (InputLayer)    [(None, 512)]                0         []                            
                                                                                                  
 Input-Segment (InputLayer)  [(None, 512)]                0         []                            
                                                                                                  
 Embedding-Token (TokenEmbe  [(None, 512, 768),           9181209   ['Input-Token[0][0]']         
 dding)                       (119547, 768)]              6                                       
                                                                                                  
 Embedding-Segment (Embeddi  (None, 512, 768)             1536      ['Input-Segment[0][0]']   

                                                                                                  
 Encoder-3-MultiHeadSelfAtt  (None, 512, 768)             2362368   ['Encoder-2-FeedForward-Norm[0
 ention (MultiHeadAttention                                         ][0]']                        
 )                                                                                                
                                                                                                  
 Encoder-3-MultiHeadSelfAtt  (None, 512, 768)             0         ['Encoder-3-MultiHeadSelfAtten
 ention-Dropout (Dropout)                                           tion[0][0]']                  
                                                                                                  
 Encoder-3-MultiHeadSelfAtt  (None, 512, 768)             0         ['Encoder-2-FeedForward-Norm[0
 ention-Add (Add)                                                   ][0]',                        
          

 Encoder-5-FeedForward-Add   (None, 512, 768)             0         ['Encoder-5-MultiHeadSelfAtten
 (Add)                                                              tion-Norm[0][0]',             
                                                                     'Encoder-5-FeedForward-Dropou
                                                                    t[0][0]']                     
                                                                                                  
 Encoder-5-FeedForward-Norm  (None, 512, 768)             1536      ['Encoder-5-FeedForward-Add[0]
  (LayerNormalization)                                              [0]']                         
                                                                                                  
 Encoder-6-MultiHeadSelfAtt  (None, 512, 768)             2362368   ['Encoder-5-FeedForward-Norm[0
 ention (MultiHeadAttention                                         ][0]']                        
 )        

                                                                                                  
 Encoder-8-FeedForward (Fee  (None, 512, 768)             4722432   ['Encoder-8-MultiHeadSelfAtten
 dForward)                                                          tion-Norm[0][0]']             
                                                                                                  
 Encoder-8-FeedForward-Drop  (None, 512, 768)             0         ['Encoder-8-FeedForward[0][0]'
 out (Dropout)                                                      ]                             
                                                                                                  
 Encoder-8-FeedForward-Add   (None, 512, 768)             0         ['Encoder-8-MultiHeadSelfAtten
 (Add)                                                              tion-Norm[0][0]',             
                                                                     'Encoder-8-FeedForward-Dropou
          

 tention-Add (Add)                                                  0][0]',                       
                                                                     'Encoder-11-MultiHeadSelfAtte
                                                                    ntion-Dropout[0][0]']         
                                                                                                  
 Encoder-11-MultiHeadSelfAt  (None, 512, 768)             1536      ['Encoder-11-MultiHeadSelfAtte
 tention-Norm (LayerNormali                                         ntion-Add[0][0]']             
 zation)                                                                                          
                                                                                                  
 Encoder-11-FeedForward (Fe  (None, 512, 768)             4722432   ['Encoder-11-MultiHeadSelfAtte
 edForward)                                                         ntion-Norm[0][0]']            
          

## Режим 1: предсказание слов, закрытых токеном [MASK] в фразе

In [4]:
sentence = 'Я пришел в [MASK] и купил [MASK].'
print(sentence)

Я пришел в [MASK] и купил [MASK].


In [5]:
sentence = sentence.replace(' [MASK] ','[MASK]'); sentence = sentence.replace('[MASK] ','[MASK]'); sentence = sentence.replace(' [MASK]','[MASK]')  # удаляем лишние пробелы
sentence = sentence.split('[MASK]')             # разбиваем строку по маске
tokens = ['[CLS]']                              # фраза всегда должна начинаться на [CLS]
# обычные строки преобразуем в токены с помощью tokenizer.tokenize(), вставляя между ними [MASK]
for i in range(len(sentence)):
    if i == 0:
        tokens = tokens + tokenizer.tokenize(sentence[i]) 
    else:
        tokens = tokens + ['[MASK]'] + tokenizer.tokenize(sentence[i]) 
tokens = tokens + ['[SEP]']                     # фраза всегда должна заканчиваться на [SEP] 

### в итоге создается последовательность токенов, начинающаяся с [CLS], содержащая токены из исходной фразы и маскированные [MASK], и заканчивающаяся на [SEP]. Эта последовательность токенов затем преобразуется в их числовые идентификаторы, дополняя нулями до длины 512.

In [6]:
token_input = tokenizer.convert_tokens_to_ids(tokens)    # Преобразование токенов в их числовые идентификаторы (token IDs):

In [7]:
token_input = token_input + [0] * (512 - len(token_input))

In [8]:
mask_input = [0]*512 # Инициализация массива mask_input длиной 512 с нулевыми значениями
for i in range(len(mask_input)): # Проход по массиву token_input и установка соответствующих значений в mask_input:
    if token_input[i] == 103:
        mask_input[i] = 1

In [9]:
seg_input = [0]*512 # Инициализация массива seg_input длиной 512 с нулевыми значениями

#### Преобразование token_input, mask_input и seg_input в массивы NumPy:

In [10]:
token_input = np.asarray([token_input])
mask_input = np.asarray([mask_input])
seg_input = np.asarray([seg_input])

#### Ок, готово. Теперь запускаем предсказание нейросети!

In [11]:
predicts = model.predict([token_input, seg_input, mask_input])[0] 
predicts = np.argmax(predicts, axis=-1)
predicts = predicts[0][:len(tokens)]    # отрезаем начало фразы, длиной как исходная фраза, чтобы отсечь случайные выбросы среди нулей дальше

1/1 [==============================] - 12s 12s/step


### Следующий код создает текстовый вывод на основе предсказаний модели BERT для позиций с [MASK].

In [12]:
out = []
# добавляем в out только слова в позиции [MASK], которые маскированы цифрой 1 в mask_input
for i in range(len(mask_input[0])):
    if mask_input[0][i] == 1:                       # [0][i], т.к. сеть возвращает batch с формой (1,512), где в первом элементе наш результат
        out.append(predicts[i]) 

out = tokenizer.convert_ids_to_tokens(out)          # индексы в текстовые токены
out = ' '.join(out)                                 # объединяем токены в строку с пробелами
out = tokenization.printable_text(out)              # в удобочитаемый текст
out = out.replace(' ##','')                         # объединяем разъединенные слова: "при ##шел" -> "пришел"

In [13]:
print('Result:', out)

Result: дом его


#### Как мы  видим, в нашем примере, для фразы "Я пришел в [MASK] и купил [MASK]." нейросеть выдала результат "дом" и "его": "Я пришел в дом и купил его". Ну, не так уж и плохо, для первого раза. Купить дом определенно лучше, чем молоко ).

# Режим 2: проверка логичности двух фраз

In [14]:
sentence_1 = 'Я пришел в магазин.'
sentence_2 = 'И купил молоко.'

print(sentence_1, '->', sentence_2)

Я пришел в магазин. -> И купил молоко.


### Следующий код выполняет токенизацию двух предложений (sentence_1 и sentence_2), преобразует их токены в числовые идентификаторы, и подготавливает входные данные для модели BERT.

In [15]:
# Токенизация первого предложения
tokens_sen_1 = tokenizer.tokenize(sentence_1)

# Токенизация второго предложения
tokens_sen_2 = tokenizer.tokenize(sentence_2)

# Формирование общего списка токенов с добавлением маркеров [CLS] и [SEP]
tokens = ['[CLS]'] + tokens_sen_1 + ['[SEP]'] + tokens_sen_2 + ['[SEP]']

In [16]:
token_input = tokenizer.convert_tokens_to_ids(tokens) # Преобразование токенов в числовые идентификаторы (token IDs)
token_input = token_input + [0] * (512 - len(token_input)) #Дополнение числовых идентификаторов нулями до длины 512

In [17]:
mask_input = [0] * 512  # Инициализация массива mask_input длиной 512 с нулевыми значениями

In [18]:
seg_input = [0]*512
len_1 = len(tokens_sen_1) + 2                   # длина первой фразы, +2 - включая начальный CLS и разделитель SEP
for i in range(len(tokens_sen_2)+1):            # +1, т.к. включая последний SEP
        seg_input[len_1 + i] = 1                # маскируем вторую фразу, включая последний SEP, единицами

# конвертируем в numpy в форму (1,) -> (1,512)
token_input = np.asarray([token_input])
mask_input = np.asarray([mask_input])
seg_input = np.asarray([seg_input])

### Пропускаем фразы через нейросеть (в этот раз результат в [1], а не в [0], как было выше)

In [19]:
predicts = model.predict([token_input, seg_input, mask_input])[1] 

1/1 [==============================] - 2s 2s/step


### И выводим вероятность того, что вторая фраза является нормальной, а не случайным набором слов

In [20]:
print('Sentence is okey:', int(round(predicts[0][0]*100)), '%')

Sentence is okey: 99 %


### На две фразы:
### Я пришел в магазин. -> И купил молоко.
### Ответ нейросети: Sentence is okey: 99 %